In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [6]:
u0 = 50.0
loss_thresh = 0.1
label = "inv_HT_tanh" 

x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [7]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        

        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) 
            
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [9]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    
    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])

    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:
max_reps = 10 #10
max_iter = 200 #75

train_loss_full = []
test_mse_full = []
test_re_full = []


lambda1_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):
    print(label)
    'Generate Training data'
    print(reps)
    torch.manual_seed(reps*36)

    train_loss = []
    test_mse_loss = []
    test_re_loss = []   

    lambda1_val = []

    N_f = 50000 #Total number of collocation points 
    N_train = 5000

    layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    nan_flag = train_model(max_iter,reps)

    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time


    lambda1_full.append(lambda1_val)

    if(nan_flag == 1):
        nan_tune.append(tune_reps)
        break

    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time,"lambda1": lambda1_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic) 


inv_HT_tanh
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 854.7452 Test MSE 858.0515976082977 Test RE 0.4931123422669901 Lambda1 -0.064031824
1 Train Loss 854.7232 Test MSE 858.068268896447 Test RE 0.493117132642091 Lambda1 -0.064144745
2 Train Loss 854.7226 Test MSE 85

76 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
77 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
78 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
79 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
80 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
81 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
82 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
83 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
84 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
85 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
86 Train L

159 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
160 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
161 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
162 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
163 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
164 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
165 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
166 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
167 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873
168 Train Loss 854.72186 Test MSE 858.0745830076004 Test RE 0.49311894694450276 Lambda1 -0.06416873


34 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
35 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
36 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
37 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
38 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
39 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
40 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
41 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
42 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
43 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
44 Train Loss 854.72

118 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
119 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
120 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
121 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
122 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
123 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
124 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
125 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
126 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
127 Train Loss 854.72046 Test MSE 858.0737249352622 Test RE 0.4931187003856127 Lambda1 -0.07837939
128 Train 

...

11 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
12 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
13 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
14 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
15 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
16 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
17 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
18 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
19 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
20 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434


93 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
94 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
95 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
96 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
97 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
98 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
99 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
100 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
101 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
102 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.0052904

175 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
176 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
177 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
178 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
179 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
180 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
181 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
182 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
183 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.005290434
184 Train Loss 854.72156 Test MSE 858.0750002233099 Test RE 0.49311906682740714 Lambda1 -0.

50 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
51 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
52 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
53 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
54 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
55 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
56 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
57 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
58 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
59 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
60 Train Loss 854.72

134 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
135 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
136 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
137 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
138 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
139 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
140 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
141 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
142 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
143 Train Loss 854.72015 Test MSE 858.074914985662 Test RE 0.4931190423351946 Lambda1 -0.015765209
144 Train 